In [1]:
%matplotlib inline
import mqbench,torch,torchvision,numpy as np,matplotlib.pyplot as plt,torchvision,torchvision.models as models,timm,timm.models as models,torch.nn as nn
from torchmetrics import ConfusionMatrix
from plot import plot2dicts
from mqbench.prepare_by_platform import prepare_by_platform
from mqbench.prepare_by_platform import BackendType
from mqbench.utils.state import enable_calibration
from mqbench.utils.state import enable_quantization
from mqbench.convert_deploy import convert_deploy
from tqdm import tqdm
from mqbench.utils.logger import logger as log
from mqbench.fake_quantize.lsq import LearnableFakeQuantize
from dataset import get_dataloader
from timm.models.swin_transformer import SwinTransformer
from logger import get_logger
from mqbench.utils.registry import DEFAULT_MODEL_QUANTIZER
from torch.fx.graph_module import GraphModule
import timm.models.swin_transformer as st
print(DEFAULT_MODEL_QUANTIZER)
logger,workdir = get_logger("softmax_quant")
log = logger # 

device = torch.device('cuda')
st.M = st.M.to(device)
mean=np.array([123.675, 116.28, 103.53])/255
std=np.array([58.395, 57.12, 57.375])/255

# dataloader = get_dataloader() #
extra_qconfig_dict = {
    'w_observer': 'MinMaxObserver',
    'a_observer': 'EMAMinMaxObserver',
    # 'a_observer': 'MSEObserver',
    'w_fakequantize': 'FixedFakeQuantize',
    'a_fakequantize': 'FixedFakeQuantize',
    # 'a_fakequantize': 'LearnableFakeQuantize',
    # 'a_fakequantize': 'FixedFakeQuantize',
    'w_qscheme': {
        'bit': 8,
        'symmetry': True,
        'per_channel': True,
        'pot_scale': False
    },
    'a_qscheme': {
        'bit': 8,
        'symmetry': True,
        'per_channel': False,
        'pot_scale': False
    }
}
logger.info(extra_qconfig_dict) #  
model = timm.create_model('swin_base_patch4_window7_224',pretrained=True).to(device) # 创建模型
# model = timm.create_model('resnet18',pretrained=True).to(device) # 创建模型
prepare_custom_config_dict = {'extra_qconfig_dict': extra_qconfig_dict}
model = prepare_by_platform(model, BackendType.Tensorrt,prepare_custom_config_dict).to(device)
ori= timm.create_model('swin_base_patch4_window7_224',pretrained=True).to(device) #
model.eval() # 进行PTQ
enable_calibration(model) # 打开校准
dataloader = get_dataloader() #
with torch.no_grad():
    for i,(img,label) in enumerate(tqdm(dataloader)):
        if i>=32:
            break
        img = img.to(device)
        model(img)

enable_quantization(model) # 打开量化，准备好模拟后台推断的量化
ori.to(device)
ori.eval()
from torchmetrics import Accuracy

from collections import defaultdict
d_ori = defaultdict(list)
d_quant = defaultdict(list)
def hook_generator(d,name):
    def hook(module,input,output):
        d[name].append((input[0].detach().cpu().numpy(),output.detach().cpu().numpy()))
        # d[name+'_output'].append(output.detach().cpu().numpy())
        return output
    return hook

for name,m in ori.named_modules():
    if name.endswith("softmax"):
        m.register_forward_hook(hook_generator(d_ori,name))

for name,m in model.named_modules():
    if name.endswith("softmax"):
        m.register_forward_hook(hook_generator(d_quant,name))

# model = ori
acc = Accuracy()
acc_ori = Accuracy()
with torch.no_grad():
    for i,(img,label) in enumerate(tqdm(dataloader)):
        if i>2:
            break
        img = img.to(device)
        # acc.update(torch.argmax(model(img),dim=-1),torch.argmax(ori(img),dim=-1))
        acc.update(model(img).detach().cpu(),label.cpu())
        acc_ori.update(ori(img).detach().cpu(),label.cpu())
# plot2dicts(d_ori,d_quant)
logger.info(f"最终的精度是：{acc.compute()}")
logger.info(f"ori_最终的精度是：{acc_ori.compute()}")

softmax_quantLog创建完成


OrderedDict()


{'w_observer': 'MinMaxObserver', 'a_observer': 'EMAMinMaxObserver', 'w_fakequantize': 'FixedFakeQuantize', 'a_fakequantize': 'FixedFakeQuantize', 'w_qscheme': {'bit': 8, 'symmetry': True, 'per_channel': True, 'pot_scale': False}, 'a_qscheme': {'bit': 8, 'symmetry': True, 'per_channel': False, 'pot_scale': False}}


[MQBENCH] INFO: Quantize model Scheme: BackendType.Tensorrt Mode: Training
[MQBENCH] INFO: Weight Quant Scheme is overrided!
[MQBENCH] INFO: Activation Quant Scheme is overrided!
[MQBENCH] INFO: Weight Qconfig:
    FakeQuantize: FixedFakeQuantize Params: {}
    Oberver:      MinMaxObserver Params: Symmetric: True / Bitwidth: 8 / Per channel: True / Pot scale: False / Extra kwargs: {}
[MQBENCH] INFO: Activation Qconfig:
    FakeQuantize: FixedFakeQuantize Params: {}
    Oberver:      EMAMinMaxObserver Params: Symmetric: True / Bitwidth: 8 / Per channel: False / Pot scale: False / Extra kwargs: {}
[MQBENCH] INFO: Replace module to qat module.
[MQBENCH] INFO: Enable observer and Disable quantize.


  2%|▏         | 32/1563 [00:12<10:03,  2.54it/s] 

[MQBENCH] INFO: Disable observer and Enable quantize.



  0%|          | 3/1563 [00:07<1:03:45,  2.45s/it]
最终的精度是：0.0
ori_最终的精度是：0.0


In [ ]:
for n,m in model.named_modules():
    if n.endswith('quantizer'):
        display(n)
        display(m.activation_post_process.min_val,m.activation_post_process.max_val)

In [ ]:
import pandas as pd
